**Grupo 2**

**Integrantes:**

1. Aviles Ferro, Richard

2. Chuquimantari Villegas, Freddy

3. Huallipe Menez, Carlo

In [2]:
#importamos librerias
import sys
import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output
#lectura de datos
import pandas as pd
import numpy as np
import random as rd
import time
#graficas
import seaborn as sns
import matplotlib.pyplot as plt

#modelo machine learning
from sklearn import model_selection,feature_selection,metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

import random

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#datos
data_original=pd.read_csv("train.csv")
data_test=pd.read_csv("test.csv")
data_copy=data_original.copy(deep=True)
data_combinada=[data_copy,data_test]

In [ ]:
list(data_original.columns.values)

**Diccionario de datos**

| Variable | Definición | 
| --- | --- |
| PassengerId | Identificador |
| Survived | Sobrevivientes |
| Pclass | Categoría social |
| Name | Nombre del pasajero |
| Sex | Sexo del pasajero |
| Age | Edad del pasajero |
| SibSp | Número de hermanos/cónyugues a bordo |
| Parch | Número de padres/hijos a bordo |
| Ticket | Código de ticket |
| Fare | Tarifa del ticket |
| Cabin | Código de cabina |
| Embarked | Puerto de embarque |

**Imputacion de datos**

In [ ]:
data_original.head(5)

In [ ]:
data_original.isnull()

In [ ]:
data_original.isnull().sum()

In [ ]:
data_test.isnull()

In [ ]:
data_test.isnull().sum()

In [ ]:
for data in data_combinada:
    #Completando datos con la mediana
    data["Age"].fillna(data["Age"].median(),inplace=True)
    #completando datos con la moda
    data["Embarked"].fillna(data["Embarked"].mode()[0],inplace=True)
    #completandon datos con la mediana
    data["Fare"].fillna(data["Fare"].median(),inplace=True)
#elimanamos datos que no son necesarios
datos_eliminar=["PassengerId","Cabin","Ticket"]
data_copy.drop(datos_eliminar,axis=1,inplace=True)

In [ ]:
data_copy.isnull().sum()

In [ ]:
data_test.isnull().sum()

In [ ]:
data_original.info()

In [ ]:
for data in data_combinada:
    data["TamFamilia"]=data["SibSp"]+data["Parch"]+1
    data["IsAlone"]=1
    data["IsAlone"].loc[data["TamFamilia"]>1]=0
    data["Titulo"]=data["Name"].str.split(", ",expand=True)[1].str.split(".",expand=True)[0]
    data["FareBin"]=pd.qcut(data["Fare"],4)
    data["AgeBin"]=pd.cut(data["Age"].astype(int), 5)
cantidad_minima=10
name_titulo=(data_copy["Titulo"].value_counts()<cantidad_minima)
#print(name_titulo)
data_copy["Titulo"]=data_copy["Titulo"].apply(lambda x: "Misc" if name_titulo.loc[x]==True else x)
data_copy["Titulo"].value_counts()

In [ ]:
data_copy

In [ ]:
data_copy.isnull().sum()

In [ ]:


etiqueta=LabelEncoder()
for data in data_combinada:
    data["SexCode"]=etiqueta.fit_transform(data["Sex"])
    data["EmbarkedCode"]=etiqueta.fit_transform(data["Embarked"])
    data["TituloCode"]=etiqueta.fit_transform(data["Titulo"])
    data["AgeBinCode"]=etiqueta.fit_transform(data["AgeBin"])
    data["FareBinCode"]=etiqueta.fit_transform(data["FareBin"])
objetivo=["Survived"]
#data1 variables de la data original
data1=['Sex','Pclass', 'Embarked', 'Titulo','SibSp', 'Parch', 'Age', 'Fare', 'TamFamilia', 'IsAlone']
data2=["SexCode","Pclass","EmbarkedCode","TituloCode",'SibSp','Parch', 'Age', 'Fare']
data3=objetivo+data2

data4=["SexCode","Pclass","EmbarkedCode","TituloCode","TamFamilia","AgeBinCode","FareBinCoce"]
data5=objetivo+data4
data1_x_bin=["SexCode","Pclass","EmbarkedCode","TituloCode","TamFamilia","AgeBinCode","FareBinCode"]
data1_xy_bin=objetivo+data1_x_bin

In [ ]:
data1_dummy=pd.get_dummies(data_copy[data1])
data1_dummy

In [ ]:
data_copy.isnull().sum()

In [ ]:
data_test.isnull().sum()

In [ ]:
data_original.describe(include="all")

In [ ]:
entrada1_x,test1_x,entrada1_y,test1_y=model_selection.train_test_split(data_copy[data2],data_copy[objetivo],random_state=0)
entrada1_x

In [ ]:
for i in data1:
    if data_copy[i].dtype!="float64":
        print(f"Correlecacion de sobrevivientes para ",i)
        print(data_copy[[i,objetivo[0]]].groupby(i,as_index=False).mean())

In [ ]:
plt.figure(figsize=[16,12])

plt.subplot(231)
plt.boxplot(x=data_copy['Fare'], showmeans = True, meanline = True)
plt.title('Fare Boxplot')
plt.ylabel('Fare ($)')

plt.subplot(232)
plt.boxplot(data_copy['Age'], showmeans = True, meanline = True)
plt.title('Edad Boxplot')
plt.ylabel('Age (Years)')

plt.subplot(233)
plt.boxplot(data_copy['TamFamilia'], showmeans = True, meanline = True)
plt.title('Family Size Boxplot')
plt.ylabel('Family Size (#)')

plt.subplot(234)
plt.hist(x = [data_copy[data_copy['Survived']==1]['Fare'], data_copy[data_copy['Survived']==0]['Fare']], 
         stacked=True, color = ['g','r'],label = ['Sobrevivio','Murio'])
plt.title('Histograma Fare by Survival')
plt.xlabel('Fare ($)')
plt.ylabel('# de pasajeros')
plt.legend()

plt.subplot(235)
plt.hist(x = [data_copy[data_copy['Survived']==1]['Age'], data_copy[data_copy['Survived']==0]['Age']], 
         stacked=True, color = ['g','r'],label = ['Sobrevivio','Murio'])
plt.title('Histograma Age by Survival')
plt.xlabel('Age (Años)')
plt.ylabel('# de pasajeros')
plt.legend()

plt.subplot(236)
plt.hist(x = [data_copy[data_copy['Survived']==1]['TamFamilia'], data_copy[data_copy['Survived']==0]['TamFamilia']], 
         stacked=True, color = ['g','r'],label = ['Sobrevivio','Murio'])
plt.title('Histograma TamFamilia vs Survival')
plt.xlabel('TamFamilia (#)')
plt.ylabel('# de pasajeros')
plt.legend()

In [ ]:
fig, saxis = plt.subplots(2, 3,figsize=(20,12))

sns.barplot(x = 'Embarked', y = 'Survived', data=data_copy, ax = saxis[0,0])
sns.barplot(x = 'Pclass', y = 'Survived', order=[1,2,3], data=data_copy, ax = saxis[0,1])
sns.barplot(x = 'IsAlone', y = 'Survived', order=[1,0], data=data_copy, ax = saxis[0,2])

sns.pointplot(x = 'FareBin', y = 'Survived',  data=data_copy, ax = saxis[1,0])
sns.pointplot(x = 'AgeBin', y = 'Survived',  data=data_copy, ax = saxis[1,1])
sns.pointplot(x = 'TamFamilia', y = 'Survived', data=data_copy, ax = saxis[1,2])

In [ ]:
fig, (axis1,axis2,axis3) = plt.subplots(1,3,figsize=(18,12))

sns.boxplot(x = 'Pclass', y = 'Fare', hue = 'Survived', data = data_copy, ax = axis1)
axis1.set_title('Comparación de Sobrevimientes: Pclass vs Fare')

sns.violinplot(x = 'Pclass', y = 'Age', hue = 'Survived', data = data_copy, split = True, ax = axis2)
axis2.set_title('Comparación de Sobrevimientes: Pclass vs Age')

sns.boxplot(x = 'Pclass', y ='TamFamilia', hue = 'Survived', data = data_copy, ax = axis3)
axis3.set_title('Comparación de Sobrevimientes: Pclass vs TamFamilia')

In [ ]:
fig, qaxis = plt.subplots(1,3,figsize=(20,12))

sns.barplot(x = 'Sex', y = 'Survived', hue = 'Embarked', data=data_copy, ax = qaxis[0])
axis1.set_title('Comparación de Sobrevimientes: Sex vs Embarked')

sns.barplot(x = 'Sex', y = 'Survived', hue = 'Pclass', data=data_copy, ax  = qaxis[1])
axis1.set_title('Comparación de Sobrevimientes: Sex vs Pclass')

sns.barplot(x = 'Sex', y = 'Survived', hue = 'IsAlone', data=data_copy, ax  = qaxis[2])
axis1.set_title('Comparación de Sobrevimientes: Sex vs IsAlone')

In [ ]:
e = sns.FacetGrid(data_copy, col = 'Embarked',height=5)
e.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', ci=95.0, palette = 'deep')
e.add_legend()


In [ ]:
a = sns.FacetGrid( data_copy, hue = 'Survived', aspect=5 )
a.map(sns.kdeplot, 'Age', shade= True )
a.set(xlim=(0 , data_copy['Age'].max()))
a.add_legend()

In [ ]:
h = sns.FacetGrid(data_copy, row = 'Sex', col = 'Pclass', hue = 'Survived',height=5)
h.map(plt.hist, 'Age', alpha = .75)
h.add_legend()

In [ ]:
pp = sns.pairplot(data_copy, hue = 'Survived', palette = 'deep', height=2.2, diag_kind = 'kde', diag_kws=dict(shade=True), plot_kws=dict(s=10) )
pp.set(xticklabels=[])

In [ ]:
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
        
    )
    
    plt.title('Matriz de correlación', y=1.05, size=15)

correlation_heatmap(data_copy)

In [ ]:
MLA = [
    #Métodos de ensamble
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Proceso gausiano
    gaussian_process.GaussianProcessClassifier(),
    
    #Modelos de Regresión
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Modelo Bayesiano
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Método de los vecinos más cercanos
    neighbors.KNeighborsClassifier(),
    
    #Máquinas de Vector Soporte
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Arboles de decisión
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Análisis discriminante lineal
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #Extreme Gradient Boosting
    XGBClassifier()    
    ]

In [ ]:
#Utilizamos model_selection.ShuffleSplit para producir índices para dividir los datos en conjuntos de entrenamiento y prueba.
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )
cv_split

In [ ]:
#Definimos los headers con los que trabajaremos
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)
MLA_compare

In [ ]:
MLA_predict = data_copy[objetivo]
MLA_compare

In [ ]:
row_index = 0
for alg in MLA:

    #definimos nombres y parámetros
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #modelo de puntuación con validación cruzada:
    cv_results = model_selection.cross_validate(alg, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #si se trata de una muestra aleatoria sin sesgo, entonces +/-3 desviaciones estándar (estándar) de la media
    #deberían capturar estadísticamente el 99,7 % de los subconjuntos
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3

    #guardamos las predicciones de MLA
    alg.fit(data_copy[data1_x_bin], data_copy[objetivo])
    MLA_predict[MLA_name] = alg.predict(data_copy[data1_x_bin])
    
    row_index+=1

    
#imprimimos y ordenamos la tabla
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare

In [ ]:
#Eficiencia de cada modelo

sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, color = 'm')

plt.title('Puntaje de precisión del algoritmo de aprendizaje automático \n')
plt.xlabel('Puntuacion de precision (%)')
plt.ylabel('Algoritmo')

In [ ]:
for  ix in data_copy.index:
    row=data_copy.loc[ix]
    
    #generamos números aleatorios en Random_Predict
    if random.random() > .5:     # numeros entre 0.0 <= x < 1.0   
        data_copy.loc[ix,'Random_Predict'] =1
    else: 
        data_copy.loc[ix,'Random_Predict'] =0    

#puntuación conjetura aleatoria de supervivencia. Utilice el método abreviado 1 = Suposición correcta y 0 = Suposición incorrecta. 
#La media de la columna será igual a la precisión.
data_copy['Random_Score'] = 0 #Asume la predicción
data_copy.loc[(data_copy['Survived'] == data_copy['Random_Predict']), 'Random_Score'] = 1 #Correcta predicción
print('Precisión del modelo Coin Flip: {:.2f}%'.format(data_copy['Random_Score'].mean()*100))

#también podemos usar la función precision_score de scikit para ahorrarnos algunas líneas de código
print('Precisión del modelo Coin Flip con SciKit: {:.2f}%'.format(metrics.accuracy_score(data_copy['Survived'], data_copy['Random_Predict'])*100))

In [ ]:
pivot_female = data_copy[data_copy.Sex=='female'].groupby(['Sex','Titulo'])['Survived'].mean()
print('Árbol de decisión de supervivencia con nodo femenino: \n',pivot_female)

pivot_male = data_copy[data_copy.Sex=='male'].groupby(['Sex','Titulo'])['Survived'].mean()
print('\n\nÁrbol de decisión de supervivencia con nodo masculino: \n',pivot_male)

In [ ]:
#https://www.iartificial.net/precision-recall-f1-accuracy-en-clasificacion/#Recall_Exhaustividad

def mytree(df):
    
   # inicializar tabla para almacenar predicciones
    Model = pd.DataFrame(data = {'Predict':[]})
    male_title = ['Master'] # títulos de sobrevivientes

    for index, row in df.iterrows():

         # Pregunta 1: ¿Estuviste en el Titanic; la mayoría murió
        Model.loc[index, 'Predict'] = 0

        # Pregunta 2: ¿Eres mujer? la mayoría sobrevivió
        if (df.loc[index, 'Sex'] == 'female'):
                  Model.loc[index, 'Predict'] = 1

              
        if ((df.loc[index, 'Sex'] == 'female') & 
            (df.loc[index, 'Pclass'] == 3) & 
            (df.loc[index, 'Embarked'] == 'S')  &
            (df.loc[index, 'Fare'] > 8)

           ):
                  Model.loc[index, 'Predict'] = 0 #Muere

        if ((df.loc[index, 'Sex'] == 'male') &
            (df.loc[index, 'Titulo'] in male_title)
            ):
            Model.loc[index, 'Predict'] = 1 #Vive
                
    return Model

Tree_Predict = mytree(data_copy)
print('Exactitud/puntuación de precisión del modelo del árbol de decisiones: {:.2f}%\n'.format(metrics.accuracy_score(data_copy['Survived'], Tree_Predict)*100))

print(metrics.classification_report(data_copy['Survived'], Tree_Predict))

In [ ]:
#Matriz de confución

import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusion',
                          cmap=plt.cm.Blues):
    """
    Esta función imprime y traza la matriz de confusión.
    La normalización se puede aplicar estableciendo 'normalize=True.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print('Matriz de confusión, sin normalización')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Valores reales')
    plt.xlabel('Valores de predicción')

# Cálculo de la matriz de confusión
cnf_matrix = metrics.confusion_matrix(data_copy['Survived'], Tree_Predict)
np.set_printoptions(precision=2)

class_names = ['Muertes', 'Sobrevivientes']
# Matriz de confusión
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Matriz de confusión, sin normalización')

# Matriz de confusión normalizada
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, 
                      title='Matriz de confusión normalizada')

In [ ]:
dtree = tree.DecisionTreeClassifier(random_state = 0)
base_results = model_selection.cross_validate(dtree, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)
#Creemos un clasificador de árbol de decisiones a partir del conjunto de entrenamiento (x, y).
dtree.fit(data_copy[data1_x_bin], data_copy[objetivo])

print('Antes de los parámetros DT: ', dtree.get_params())
print("Antes de la prueba DT con media de puntuación bin: {:.2f}". format(base_results['test_score'].mean()*100))
print("Antes de la prueba DT con puntuación de bin 3 * std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))

print('-'*10)

#Entrenamos la data
param_grid = {'criterion': ['gini', 'entropy'], 

              'max_depth': [2,4,6,8,10,None], 

              'random_state': [0] 
             }

tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
tune_model.fit(data_copy[data1_x_bin], data_copy[objetivo])


print('Después de los parámetros DT: ', tune_model.best_params_)

print("Después de la prueba DT con media de puntuación bin: {:.2f}". format(tune_model.cv_results_['mean_test_score'][tune_model.best_index_]*100))
print("Después de la prueba DT con puntuación bin 3 * std: +/- {:.2f}". format(tune_model.cv_results_['std_test_score'][tune_model.best_index_]*100*3))
print('-'*10)

In [ ]:
#Imprimimos columnas sin entrenar
print('Antes de la prueba DT RFE: ', data_copy[data1_x_bin].shape) 
print('Antes de la prueba DT RFE Formación Columnas Antiguo: ', data_copy[data1_x_bin].columns.values)


print("Antes de la prueba DT RFE con la media de la puntuación del intervalo: {:.2f}". format(base_results['test_score'].mean()*100))
print("Antes de la prueba DT RFE con puntuación bin 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
print('-'*10)

#selección de columnas y entrenamiento
dtree_rfe = feature_selection.RFECV(dtree, step = 1, scoring = 'accuracy', cv = cv_split)
dtree_rfe.fit(data_copy[data1_x_bin], data_copy[objetivo])

#transformamos x&y en características reducidas para adaptarse al nuevo modelo
X_rfe = data_copy[data1_x_bin].columns.values[dtree_rfe.get_support()]
rfe_results = model_selection.cross_validate(dtree, data_copy[X_rfe], data_copy[objetivo], cv  = cv_split)

print('Después de la forma de entrenamiento DT RFE Nuevo: ', data_copy[X_rfe].shape) 
print('Después de las columnas de entrenamiento DT RFE Nuevo: ', X_rfe)

print("Después de la prueba DT RFE con promedio de puntaje de bin: {:.2f}". format(rfe_results['test_score'].mean()*100))
print("Después de la prueba DT RFE con puntuación bin 3*std: +/- {:.2f}". format(rfe_results['test_score'].std()*100*3))
print('-'*10)


#Entrenamos el nuevo modelo con el metodo fit
rfe_tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(criterion='entropy',
                                            min_samples_split=20,
                                            min_samples_leaf=5,
                                            max_depth = 4,
                                            class_weight={1:3.5}), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
rfe_tune_model.fit(data_copy[X_rfe], data_copy[objetivo])

print('Después de los parámetros elegidos DT RFE: ', rfe_tune_model.best_params_)

print("Después de la prueba DT RFE con promedio de puntaje de bin: {:.2f}". format(rfe_tune_model.cv_results_['mean_test_score'][tune_model.best_index_]*100))
print("Después de la prueba DT RFE con puntuación bin 3*std: +/- {:.2f}". format(rfe_tune_model.cv_results_['std_test_score'][tune_model.best_index_]*100*3))
print('-'*10)

In [ ]:
#Matriz de correlación

correlation_heatmap(MLA_predict)

In [ ]:
vote_est = [
    #Meteodos de Ensable
    ('ada', ensemble.AdaBoostClassifier()),
    ('bc', ensemble.BaggingClassifier()),
    ('etc',ensemble.ExtraTreesClassifier()),
    ('gbc', ensemble.GradientBoostingClassifier()),
    ('rfc', ensemble.RandomForestClassifier()),

    #Proceso gausiano
    ('gpc', gaussian_process.GaussianProcessClassifier()),
    
    #Modelos de Regresión
    ('lr', linear_model.LogisticRegressionCV()),
    
    #Modelo Bayesiano
    ('bnb', naive_bayes.BernoulliNB()),
    ('gnb', naive_bayes.GaussianNB()),
    
    #Método de los vecinos más cercanos
    ('knn', neighbors.KNeighborsClassifier()),
    
    #Máquinas de Vector Soporte
    ('svc', svm.SVC(probability=True)),
    
    #Extreme Gradient Boosting
   ('xgb', XGBClassifier())

]


#Reglas de mayoría
vote_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
vote_hard_cv = model_selection.cross_validate(vote_hard, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)
vote_hard.fit(data_copy[data1_x_bin], data_copy[objetivo])


print("Prueba de votación de reglas de mayoría w/bin con promedio: {:.2f}". format(vote_hard_cv['test_score'].mean()*100))
print("Prueba de votación de reglas de mayoría w/bin con 3*std: +/- {:.2f}". format(vote_hard_cv['test_score'].std()*100*3))
print('-'*10)


#Probabilidades ponderadas
vote_soft = ensemble.VotingClassifier(estimators = vote_est , voting = 'soft')
vote_soft_cv = model_selection.cross_validate(vote_soft, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)
vote_soft.fit(data_copy[data1_x_bin], data_copy[objetivo])


print("Prueba de votación de probabilidades ponderadas w/bin con promedio:". format(vote_soft_cv['test_score'].mean()*100))
print("Prueba de votación de probabilidades ponderadas w/bin con 3*std: +/- {:.2f}". format(vote_soft_cv['test_score'].std()*100*3))
print('-'*10)

In [ ]:
#Organización de hiperparámetros con GridSearchCV

grid_n_estimator = [10, 50, 100, 300]
grid_ratio = [.1, .25, .5, .75, 1.0]
grid_learn = [.01, .03, .05, .1, .25]
grid_max_depth = [2, 4, 6, 8, 10, None]
grid_min_samples = [5, 10, .03, .05, .10]
grid_criterion = ['gini', 'entropy']
grid_bool = [True, False]
grid_seed = [0]


grid_param = [
            [{
            #AdaBoostClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
            'n_estimators': grid_n_estimator,
            'learning_rate': grid_learn,
            'random_state': grid_seed
            }],
       
    
            [{
            #BaggingClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier
            'n_estimators': grid_n_estimator,
            'max_samples': grid_ratio,
            'random_state': grid_seed
             }],

    
            [{
            #ExtraTreesClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier
            'n_estimators': grid_n_estimator,
            'criterion': grid_criterion,
            'max_depth': grid_max_depth,
            'random_state': grid_seed
             }],


            [{
            #GradientBoostingClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier
            'learning_rate': [.05],
            'n_estimators': [300],
            'max_depth': grid_max_depth,
            'random_state': grid_seed
             }],

    
            [{
            #RandomForestClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
            'n_estimators': grid_n_estimator,
            'criterion': grid_criterion,
            'max_depth': grid_max_depth,
            'oob_score': [True],
            'random_state': grid_seed
             }],
    
            [{
            #GaussianProcessClassifier
            'max_iter_predict': grid_n_estimator, #por defecto: 100
            'random_state': grid_seed
            }],
        
    
            [{
            #LogisticRegressionCV - http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV
            'fit_intercept': grid_bool,
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'random_state': grid_seed
             }],
            
    
            [{
            #BernoulliNB - http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB
            'alpha': grid_ratio,
             }],

    
            #GaussianNB - 
            [{}],
    
    
            [{
            #KNeighborsClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
            'n_neighbors': [1,2,3,4,5,6,7],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
            }],
            
    
            [{
            #SVC - http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
            'C': [1,2,3,4,5],
            'gamma': grid_ratio,
            'decision_function_shape': ['ovo', 'ovr'],
            'probability': [True],
            'random_state': grid_seed
             }],

    
            [{
            #XGBClassifier - http://xgboost.readthedocs.io/en/latest/parameter.html
            'learning_rate': grid_learn, 
            'max_depth': [1,2,4,6,8,10],
            'n_estimators': grid_n_estimator, 
            'seed': grid_seed  
             }]   
        ]


start_total = time.perf_counter()
for clf, param in zip (vote_est, grid_param):

    start = time.perf_counter()        
    best_search = model_selection.GridSearchCV(estimator = clf[1], param_grid = param, cv = cv_split, scoring = 'roc_auc')
    best_search.fit(data_copy[data1_x_bin], data_copy[objetivo])
    run = time.perf_counter() - start

    best_param = best_search.best_params_
    print('El mejor parámetro para {} es {} con un tiempo de ejeción de {:.2f} segundos.'.format(clf[1].__class__.__name__, best_param, run))
    clf[1].set_params(**best_param) 


run_total = time.perf_counter() - start_total
print('El tiempo total de optimización fue {:.2f} minutos.'.format(run_total/60))

print('-'*10)

In [ ]:
#Elección fija o reglas de mayoría con con hiperparámetros ajustados
grid_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
grid_hard_cv = model_selection.cross_validate(grid_hard, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)
grid_hard.fit(data_copy[data1_x_bin], data_copy[objetivo])


print("Elección fija con prueba de hiperparámetros organizados con la media: {:.2f}". format(grid_hard_cv['test_score'].mean()*100))
print("Elección fija con prueba de hiperparámetros organizados con 3*std: +/- {:.2f}". format(grid_hard_cv['test_score'].std()*100*3))
print('-'*10)

#Elección flexible o probabilidades ponderadas con hiperparámetros ajustados
grid_soft = ensemble.VotingClassifier(estimators = vote_est , voting = 'soft')
grid_soft_cv = model_selection.cross_validate(grid_soft, data_copy[data1_x_bin], data_copy[objetivo], cv  = cv_split)
grid_soft.fit(data_copy[data1_x_bin], data_copy[objetivo])


print("Elección flexible con prueba de hiperparámetros organizados con la media: {:.2f}". format(grid_soft_cv['test_score'].mean()*100))
print("Elección flexible con prueba de hiperparámetros organizados con 3*std: +/- {:.2f}". format(grid_soft_cv['test_score'].std()*100*3))
print('-'*10)

In [ ]:
#Preparamos la data para el modelo
print(data_test.info())
print("-"*10)

#Mostramos los resultados y los convertimos a enteros
data_test['Survived'] = mytree(data_test).astype(int)
data_test

In [ ]:
data_test.head(10)

In [ ]:
submit = data_test[['PassengerId','Survived']]
submit.to_csv("resultado_prediccion.csv", index=False)

print('Conteo de datos validados: \n', data_test['Survived'].value_counts(normalize = True))
submit